In [1]:

import os
import pickle
import pandas as pd
import sys
import sys
from memory_profiler import profile
import pandas as pd
import numpy as np
sys.path.append('/home/ak/Documents/Research/PaperCode/MultiKernelLearning')
sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
import jsonpickle
from collections import defaultdict
from sklearn.svm import SVC
from collections import OrderedDict
import os
from sklearn.metrics import accuracy_score
import torch


from fileutils import new_feature_utils as nfu
from fileutils.new_feature_utils import CreateMarketFeatures
import multiprocessing
import mkl_data_processing as mkldp
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
import matplotlib as mpl
import matplotlib.pyplot as plt
styleFiles = os.listdir(mpl.get_configdir())
styleFileIdx = 0

plt.style.use(os.path.join(mpl.get_configdir(), styleFiles[styleFileIdx]))


Bad key "axes.titlelocation" on line 28 in
/home/ak/Envs/resrPyth3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/plot_style.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def open_pickle_filepath(pickle_file):
    pickle_to_file = pickle.load(open(pickle_file, "rb"), encoding='latin1')

    return pickle_to_file

In [5]:
files_location = '/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSPredictions/SimpleMKL/'
symbols = sorted(os.listdir(files_location))
#processed results path
processed_location = '/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSProcessed'

In [12]:
symbolIDX =  1 #pick a symbol
label_idx = 5 #ick a label
label_string = 'Label_idx:_'+str(label_idx)
print('you are doing symbol', symbols[symbolIDX], 'and label', label_string)
# now we take out the files for the symbol 
symbol_path = os.path.join(files_location,str(sorted(symbols)[symbolIDX]))
symbolFiles = os.listdir(os.path.join(files_location,str(sorted(symbols)[symbolIDX])))
# now isolate all the label+ symbol combos
symbolLabelFiles = [f for f in symbolFiles if str(label_string) in f]
path_to_store = os.path.join(processed_location, symbols[symbolIDX])
number_of_files = len(symbolLabelFiles) 

# this is the number of files that correspond to the label/symbol combo

print('total files for the symbol:', len(symbolFiles))
print('and for this label')
print(number_of_files)

you are doing symbol APF.L and label Label_idx:_5
total files for the symbol: 3192
and for this label
0


In [313]:
path_to_store

'/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSProcessed/SDR.L'

In [298]:
dates_chunks_keys = dict()
chunks_list =list()
keys_list = list()

for file_to_load_idx in range(0,number_of_files):
    # go through all of the files that are a combination of symbols and labels
    file_to_load_path = os.path.join(os.path.join(files_location, str(symbols[symbolIDX])), symbolLabelFiles[file_to_load_idx])
    if os.path.getsize(file_to_load_path)> 0:
        # extract the key - these are the symbol feature path keys in the  main code
        file_keys = list(open_pickle_filepath(file_to_load_path).keys()) # keys
        keys_list.append(file_keys[0])
        dates_chunks_keys[file_to_load_idx] = file_keys
        chunks_list.append(open_pickle_filepath(file_to_load_path)[file_keys[0]])
    else:
        continue

        

In [299]:
dates_df = pd.DataFrame((np.unique(list(dates_chunks_keys.values()))))
list_of_dfs_across_days = list()
for chunk_list_item in range(0,len(chunks_list)-1):
    chunk_list_item_dict = chunks_list[chunk_list_item]
    chunk_list_item_keys = list(chunk_list_item_dict.keys())
    list_of_dicts=list()
    for key in chunk_list_item_keys:
        list_of_dicts.append(pd.DataFrame(chunk_list_item_dict[key]))
    list_of_dfs_across_days.append(pd.concat(list_of_dicts,axis=1).median(axis=1))


In [300]:
df =pd.concat(list_of_dfs_across_days, axis =1)
df.median(axis=1)

Hamming Loss    0.670
accuracy        0.330
f1- macro       0.230
f1- micro       0.330
f1- weighted    0.245
precision       0.290
recall          0.330
dtype: float64

In [301]:
#df.to_csv('file_name.csv
location_for_df = os.path.join(path_to_store, symbols[symbolIDX] +"_Label_"+str(label_idx)+"_all_chunks.csv")
df.to_csv(location_for_df, index= True)
location_for_dates_df = os.path.join(path_to_store, symbols[symbolIDX] +"_Label_"+str(label_idx)+"_dates.csv")
dates_df.to_csv(location_for_dates_df, index = True)

In [302]:
df_test = pd.read_csv(location_for_df, index_col=0)
df_test.median(axis=1)

Hamming Loss    0.670
accuracy        0.330
f1- macro       0.230
f1- micro       0.330
f1- weighted    0.245
precision       0.290
recall          0.330
dtype: float64

# to do:
1. write a loop to go through all the labels for each symbol and store them in a seperate file
2. concentrate only on the symbols we know we have labels
3. pull the file from Single Kernel and compare them.
4. re-do the labels and the plots for the symbols that you are picking and describe them

In [160]:

# Path
def create_path(parent_dir, symbol):
    path = os.path.join(parent_dir, symbol)
    if os.path.isdir(path)==False:
        # Create the directory
    
        os.mkdir(path)
        print("Directory '%s' created" %path)
    else:
        print("Directory Exist")
        pass

In [161]:
symbols[symbolIDX]

'APF.L'

In [64]:
for symbolIDX in range(0, len(symbols)):
    create_path(processed_location, symbols[symbolIDX])

Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory Exist
Directory '/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSProcessed/WPP.L' created


40